In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sys
import shap
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
plt.style.use('fivethirtyeight')
%matplotlib inline

In [ ]:
sessions = pd.read_json("../data/raw/second/sessions.jsonl", lines=True)
buy_sessions = sessions[sessions['event_type'] == 'BUY_PRODUCT'].copy()
buy_sessions['product_id'] = buy_sessions['product_id'].astype(str)
buy_sessions['date']=buy_sessions['timestamp'].dt.date
buy_sessions.drop(columns=['user_id', 'event_type', 'offered_discount','session_id','timestamp'], inplace=True)
df = buy_sessions.groupby(by=['date','product_id']).count().unstack(fill_value=0).stack().rename(columns={'purchase_id': 'amount'}).reset_index()
df['product_id'] = df['product_id'].astype(str)

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df=df.groupby(by=[pd.Grouper(key='date', freq='W'), pd.Grouper(key='product_id')]).sum()

In [ ]:
print(df)

In [ ]:
products = pd.read_json("../data/raw/second/products.jsonl", lines=True)
del products['product_name']
del products['category_path']
products['product_id']=products['product_id'].astype(str)
products.index=products['product_id']
df=df.join(products,on='product_id')

In [ ]:
df.index = df.index.get_level_values(0)

In [ ]:
df

In [ ]:
df['time'] = np.arange(len(df.index))

In [ ]:
df['lag']=df['amount'].shift(1)
df=df.fillna(0)

In [ ]:
split_date = pd.to_datetime('2021-09-01')
train = df.loc[df.index <= split_date].copy()
test = df.loc[df.index > split_date].copy()

In [ ]:
del train['product_id']
del test['product_id']

In [ ]:
X_train=train.drop('amount', axis=1)
y_train=train['amount']
              
X_test=test.drop('amount', axis=1)
y_test=test['amount']

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
test.head(10)

In [ ]:
test['amount_prediction'] = model.predict(X_test)
df_all = pd.concat([train, test], sort=False)
_ = df_all[['amount','amount_prediction']].plot(figsize=(15, 5))